In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.gridspec as gridspec
from astropy.io import fits


In [12]:
path='/Users/baotong/data_GalDisc/data/'
allinfo=pd.read_excel(path+'match_e_xmm/'+'e_xmmdr14s_match_all_starinfo.xlsx',sheet_name='Sheet1')
print(allinfo['xmm_index'])
xmmdr14slist = fits.open(path + 'xmmdr14s/GalDisc_4xmmdr14s_new_cleaned.fits')[1].data
ncontrib = xmmdr14slist['N_CONTRIB']
realsrc = xmmdr14slist[np.where(ncontrib > 0)]
xmmflux=realsrc['EP_FLUX'][allinfo['xmm_index']]
xmmfluxerr=realsrc['EP_FLUX_ERR'][allinfo['xmm_index']]
hr1=realsrc['EP_HR1'][allinfo['xmm_index']]
hr2=realsrc['EP_HR2'][allinfo['xmm_index']]
hr3=realsrc['EP_HR3'][allinfo['xmm_index']]
hr4=realsrc['EP_HR4'][allinfo['xmm_index']]

0      6042
1      9149
2      4982
3      9678
4      9676
       ... 
177    6616
178    6466
179    5660
180    5849
181    6339
Name: xmm_index, Length: 182, dtype: int64


In [14]:
modified_df = allinfo.copy()
modified_df['xmmflux'] = xmmflux
modified_df['xmmfluxerr'] = xmmfluxerr
modified_df['hr1'] = hr1
modified_df['hr2'] = hr2
modified_df['hr3'] = hr3
modified_df['hr4'] = hr4

# Write the DataFrame with new columns to a new sheet in the existing Excel file
with pd.ExcelWriter(path + 'match_e_xmm/modified_starinfo.xlsx', engine='xlsxwriter') as writer:
    # Write the original data first (Sheet1)
    allinfo.to_excel(writer, sheet_name='Sheet1', index=False)
    modified_df.to_excel(writer, sheet_name='modified', index=False)

print("Data successfully written to the new 'modified' sheet.")

Data successfully written to the new 'modified' sheet.


In [52]:
import pandas as pd
from openpyxl import load_workbook

def filter_closest_gaia_match(input_file, sheet_name='Sheet1', new_sheet_name='ClosestMatches'):
    """
    Filters the closest Gaia match for each unique source and saves to a new sheet in the original Excel file.
    
    Keeps rows with NaN values in 'gaia_separation_arcsec' for unmatched sources.
    Prioritizes rows where 'hamstarindex' > 0 by keeping the row where 'gaia_index' matches 'CTP_ID'.
    If no matching row is found, keeps the row with the minimum 'gaia_separation_arcsec' ignoring NaNs.

    :param input_file: Path to the input Excel file.
    :param sheet_name: Name of the sheet to read from. Default is 'Sheet1'.
    :param new_sheet_name: Name of the new sheet to save filtered results. Default is 'ClosestMatches'.
    """
    # Read the specified sheet from the Excel file
    df = pd.read_excel(input_file, sheet_name=sheet_name, engine='openpyxl')

    # Define a function to filter rows based on the criteria
    def filter_group(group):
        # If 'hamstarindex' > 0, try to select the row where 'gaia_index' matches 'CTP_ID'
        if (group['hamstarindex'] > 0).any():
            matching_row = group[group['gaia_index'] == group['CTP_ID']]
            if not matching_row.empty:
                return matching_row
            # If no matching row is found, fall back to the row with minimum 'gaia_separation_arcsec' ignoring NaNs
            if group['gaia_separation_arcsec'].isna().all():
                return group.iloc[[0]]
            non_nan_group = group.dropna(subset=['gaia_separation_arcsec'])
            if not non_nan_group.empty:
                return non_nan_group.loc[[non_nan_group['gaia_separation_arcsec'].idxmin()]]
        else:
            # If all 'gaia_separation_arcsec' are NaN, keep the first row
            if group['gaia_separation_arcsec'].isna().all():
                return group.iloc[[0]]
            # Otherwise, return the row with the minimum 'gaia_separation_arcsec' ignoring NaNs
            non_nan_group = group.dropna(subset=['gaia_separation_arcsec'])
            return non_nan_group.loc[[non_nan_group['gaia_separation_arcsec'].idxmin()]]

    # Apply the filter to each group of 'e_id' and ensure the result is flattened
    closest_matches = df.groupby('e_id', as_index=False, group_keys=False).apply(filter_group)
    # Convert the dataframe back to string for consistency
    closest_matches = closest_matches.astype(str)

    # Load the workbook to append the new sheet
    with pd.ExcelWriter(input_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        closest_matches.to_excel(writer, sheet_name=new_sheet_name, index=False)


In [54]:
path='/Users/baotong/data_GalDisc/data/match_e_xmm/'
input_file = path+'matched_gaia_results_optimized_4sec.xlsx'
filter_closest_gaia_match(input_file)